# 基于千帆与 FLAML 的模型配置自动推荐

千帆 SDK 提供了模型配置自动推荐功能，用户只需要提供目标场景的数据集和评估器，指定搜索空间后，SDK 就可以自动推荐出最佳的模型配置。

与此同时，FLAML 是一个开源的 AutoML 库，提供了许多优秀的参数推荐算法，可以更高效地去寻找最佳的模型配置。

然而，由于 FLAML 本身并不支持千帆 SDK 的数据集和评估器，这需要额外的开发工作。对此，千帆 SDK 为 FLAML 提供了适配层，可以快速基于千帆和 FLAML 的算法实现模型配置自动推荐。

In [ ]:
!pip install qianfan
!pip install flaml[blendsearch]

In [ ]:
import os
import qianfan

os.environ["QIANFAN_ACCESS_KEY"] = "your_access_key"
os.environ["QIANFAN_SECRET_KEY"] = "your_secret_key"
# 由于后续在调优配置的过程中会并发请求模型，建议限制 QPS 和重试次数，避免调用失败
os.environ["QIANFAN_QPS_LIMIT"] = "3"
os.environ["QIANFAN_LLM_API_RETRY_COUNT"] = "5"

## 准备工作

与直接使用千帆 SDK 的参数推荐功能相同，我们需要先准备：

- 数据集 Dataset：根据目标场景准备一定量的数据
- 评估方式 Evaluator：根据目标场景，选择待优化的指标，并提供评估函数

数据集使用的是千帆 SDK 中提供的 Dataset 模块，可以直接加载本地的数据集文件，也可以使用平台上预置的或者自行上传的数据集，具体加载方式参考 [文档](https://github.com/baidubce/bce-qianfan-sdk/blob/main/docs/dataset.md)。

评估采用的 SDK 提供的 Evaluator 模块，基于 Evaluator 实现 evaluate 方法即可，关于如何实现 Evaluator 可以参考 [该cookbook](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/evaluation/local_eval_with_qianfan.ipynb)。

如下加载了本地的一个数据集，并实现了一个利用大模型评分实现评估的 Evaluator，如果你对基础的使用方式还不熟悉，可以参考 [该cookbook](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/autotuner/tune.ipynb)，此处省略相关的介绍。

In [1]:
from qianfan.dataset import Dataset
from qianfan.evaluation.evaluator import LocalEvaluator
from qianfan import ChatCompletion
from qianfan.common.prompt.prompt import Prompt
from qianfan.utils.pydantic import Field

from typing import Optional, Union, Any, Dict, List
import re
import json

dataset = Dataset.load(
    data_file="./example.jsonl",
    organize_data_as_group=False,
    input_columns=["prompt"],
    reference_column="response",
)

class LocalJudgeEvaluator(LocalEvaluator):
    model: Optional[ChatCompletion] = Field(default=None, description="model object")
    cache: Dict[str, Any] = Field(default={}, description="cache for evaluation")
    eval_prompt: Prompt = Field(
        default=Prompt(
            template="""你需要扮演一个裁判的角色，对一段角色扮演的对话内容进行打分，你需要考虑这段文本中的角色沉浸度和对话文本的通畅程度。你可以根据以下规则来进行打分，你可以阐述你对打分标准的理解后再给出分数：
                "4":完全可以扮演提问中的角色进行对话，回答完全符合角色口吻和身份，文本流畅语句通顺
                "3":扮演了提问中正确的角色，回答完全符合角色口吻和身份，但文本不流畅或字数不满足要求
                "2":扮演了提问中正确的角色，但是部分语句不符合角色口吻和身份，文本流畅语句通顺
                "1":能够以角色的口吻和身份进行一部分对话，和角色设定有一定偏差，回答内容不流畅，或不满足文本字数要求
                "0":扮演了错误的角色，没有扮演正确的角色，角色设定和提问设定差异极大，完全不满意
                你的回答需要以json代码格式输出：
                ```json
                {"modelA": {"justification": "此处阐述对打分标准的理解", "score": "此处填写打分结果"}}
                ```

                现在你可以开始回答了：
                问题：{{input}}
                ---
                modelA回答：{{output}}
                ---""",
            identifier="{{}}",
        ),
        description="evaluation prompt",
    )

    class Config:
        arbitrary_types_allowed = True

    def evaluate(
        self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
    ) -> Dict[str, Any]:
        score = 0
        try:
            # 渲染评估用的 prompt，传入输入、模型输出和参考答案
            p, _ = self.eval_prompt.render(
                **{
                    "input": "\n".join([i["content"] for i in input[1:]]),
                    "output": output,
                    "expect": reference,
                }
            )
            # 利用 cache 避免对同一结果反复进行评估，提升效率
            if p in self.cache:
                model_output = self.cache[p]
                score = float(model_output["modelA"]["score"])
            else:
                # 请求模型进行评估
                r = self.model.do(messages=[{"role": "user", "content": p}], temperature=0.01)
                content = r["result"]
                model_output = content
                # 提取出 json 格式的评估结果
                regex = re.compile("\`\`\`json(.*)\`\`\`", re.MULTILINE | re.DOTALL)
    
                u = regex.findall(content)
    
                if len(u) == 0:
                    score = 0
                else:
                    model_output = json.loads(u[0])
                    score = float(model_output["modelA"]["score"])
                    self.cache[p] = model_output
        except Exception as e:
            score = 0
        # 返回评估结果，这里字段需与后续推荐配置时设定的评估字段一致
        return {"score": score}

    def summarize(self, metric_dataset: Dataset) -> Optional[Dict[str, Any]]:
        score_sum = 0
        count = 0

        for line in metric_dataset.list():
            score_sum += line["score"]
            count += 1

        return {"score": score_sum / float(count)}

local_evaluator = LocalJudgeEvaluator(
    model=ChatCompletion(model="ERNIE-Bot-4")
)

[INFO] [04-25 15:25:52] dataset.py:408 [t:139839157186816]: no data source was provided, construct
[INFO] [04-25 15:25:52] dataset.py:276 [t:139839157186816]: construct a file data source from path: ./example.jsonl, with args: {'input_columns': ['prompt'], 'reference_column': 'response'}
[INFO] [04-25 15:25:52] file.py:298 [t:139839157186816]: use format type FormatType.Jsonl
[INFO] [04-25 15:25:52] utils.py:416 [t:139839157186816]: need create cached arrow file for /root/work/bce-qianfan-sdk/cookbook/autotuner/example.jsonl
[INFO] [04-25 15:25:52] utils.py:461 [t:139839157186816]: start to write arrow table to /root/.qianfan_cache/dataset/root/work/bce-qianfan-sdk/cookbook/autotuner/example.arrow
[INFO] [04-25 15:25:52] utils.py:473 [t:139839157186816]: writing succeeded
[INFO] [04-25 15:25:52] utils.py:347 [t:139839157186816]: start to get memory_map from /root/.qianfan_cache/dataset/root/work/bce-qianfan-sdk/cookbook/autotuner/example.arrow
[INFO] [04-25 15:25:52] utils.py:275 [t:13

在开始调优前，我们还需要选择调优的算法和相关配置，这部分使用的是 FLAML 提供的 Searcher，我们需要准备一个 searcher，如下以 BlendSearch 为例

In [2]:
from flaml import BlendSearch, tune

blendsearch = BlendSearch(
    metric="score",  # 指标，与 Evalutor 返回结果对应
    mode="max",
    space={          # 搜索空间
        "temperature": tune.uniform(0.01, 0.99),
        "model": tune.choice(
            [
                "ERNIE-Speed",
                "ERNIE-Bot-turbo",
            ]
        ),
    },
    time_budget_s=600,       # 时间约束
    cost_attr="total_cost",  # 设置成本的 key，使用千帆 SDK 时该 key 为 total_cost
    cost_budget=20,          # 成本约束，单位为 元
)


You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2024-04-25 15:42:06,661] A new study created in memory with name: optuna


## 开始调优

之后我们就可以传入参数开始寻找最有的模型配置了

In [5]:
from qianfan.extensions.flaml.autotuner import run

context = await run(
    searcher=blendsearch,
    dataset=dataset,
    evaluator=local_evaluator,
    # 以下均为可选参数
    repeat=5,            # 重复推理次数，用于减少大模型输出随机性对结果准确性的干扰
    max_turn=10,         # 设定最大尝试次数
    # max_time=3600,     # 设定最大尝试时间，单位为秒
    log_dir= "./log",    # 日志目录
)

[INFO] [04-25 15:44:48] launcher.py:108 [t:139839157186816]: turn 0 started...
[INFO] [04-25 15:44:48] launcher.py:109 [t:139839157186816]: suggested config list: [{'temperature': 0.765894230401411, 'model': 'ERNIE-Bot-turbo'}]
[INFO] [04-25 15:44:48] dataset.py:994 [t:139839157186816]: list local dataset data by None
[INFO] [04-25 15:48:09] launcher.py:114 [t:139839157186816]: config: {'temperature': 0.765894230401411, 'model': 'ERNIE-Bot-turbo'}, metrics: {'score': 2.21, 'avg_prompt_tokens': 754.8, 'avg_completion_tokens': 106.42, 'avg_total_tokens': 861.22, 'avg_req_latency': 2.439878113121231, 'avg_tokens_per_second': 352.97664886147874, 'avg_cost': 0.0029029200000000007, 'total_cost': 0.29029200000000005, 'success_rate': 1.0, 'total_time': 201.03477883338928}
[INFO] [04-25 15:48:09] launcher.py:108 [t:139839157186816]: turn 1 started...
[INFO] [04-25 15:48:09] launcher.py:109 [t:139839157186816]: suggested config list: [{'temperature': 0.7438278048878396, 'model': 'ERNIE-Speed'}]


可以通过如下方式获取最佳的参数

In [6]:
context.best

{'temperature': 0.2041016074644315, 'model': 'ERNIE-Speed'}

或者通过如下方式获取每次尝试的配置及结果

In [7]:
for turn in context.history:
    for trial in turn:
        metrics = trial.metrics
        config = trial.config
        print("{}\t{}\t{}".format(config['model'], config['temperature'], metrics['score']))

ERNIE-Bot-turbo	0.765894230401411	2.21
ERNIE-Speed	0.7438278048878396	2.67
ERNIE-Speed	0.2041016074644315	3.21
ERNIE-Bot-turbo	0.09657301789053005	2.56
ERNIE-Speed	0.3425908632261716	2.85
ERNIE-Bot-turbo	0.013869301001356162	2.385
ERNIE-Speed	0.6102755454928004	2.75
ERNIE-Speed	0.9094186400626846	2.44
ERNIE-Bot-turbo	0.14932664664949644	2.475
ERNIE-Bot-turbo	0.06561235170269139	2.31


至此我们就完成了基于千帆 SDK 和 FLAML 的模型配置自动推荐。